In [ ]:
#converting json data to dpacy format
import json
import random
import logging
from sklearn.metrics import classification_report
from sklearn.metrics import precision_recall_fscore_support
from spacy.gold import GoldParse
from spacy.scorer import Scorer
from sklearn.metrics import accuracy_score
def convert_dataturks_to_spacy(dataturks_JSON_FilePath):
    try:
        training_data = []
        lines=[]
        with open(dataturks_JSON_FilePath, 'r') as f:
#             lines = f.readlines()
            lines = json.load(f)
        for line in lines:
#             data = json.loads(line)
            data=line.items()
            for key, value in data:
                if(key=='content'):
                    text = value
            entities = []
            for key, value in data:
                if(key=='entities'):
                    for annotation in range(len(value)):
                        #only a single point in text annotation.
                        point = value[annotation][0]
                        labels = value[annotation][2]
                        # handle both list of labels or a single label.
                        if not isinstance(labels, list):
                            labels = [labels]
                        for label in labels:
                            #dataturks indices are both inclusive [start, end] but spacy is not [start, end)
                            entities.append((value[annotation][0], value[annotation][1]+1 ,label))

            training_data.append((text, {"entities" : entities}))

        return training_data
    except Exception as e:
        logging.exception("Unable to process " + dataturks_JSON_FilePath + "\n" + "error = " + str(e))
        return None

In [ ]:
import spacy
TRAIN_DATA = convert_dataturks_to_spacy("../input/new-train/output.json")
nlp = spacy.blank('en')

In [ ]:
print(TRAIN_DATA)

In [ ]:
if 'ner' not in nlp.pipe_names:
        ner = nlp.create_pipe('ner')
        nlp.add_pipe(ner, last=True)

In [ ]:
# add labels
for _, annotations in TRAIN_DATA:
    for ent in annotations.get('entities'):
        ner.add_label(ent[2])

In [ ]:
import spacy

################### Train Spacy NER.###########
def train_spacy():
    
    # get names of other pipes to disable them during training
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
    with nlp.disable_pipes(*other_pipes):  # only train NER
        optimizer = nlp.begin_training()
        for itn in range(30):
            print("Statring iteration " + str(itn))
            random.shuffle(TRAIN_DATA)
            losses = {}
            for text, annotations in TRAIN_DATA:
                nlp.update(
                    [text],  # batch of texts
                    [annotations],  # batch of annotations
                    drop=0.5,  # dropout - make it harder to memorise data
                    sgd=optimizer,  # callable to update weights
                    losses=losses)
            print(losses)

In [ ]:
train_spacy()

In [ ]:
from pathlib import Path
output_dir=("C:\\Users\\ishik\\Downloads\\Spacy_model")

In [ ]:
# save model to output directory
if output_dir is not None:
    output_dir = Path(output_dir)
    if not output_dir.exists():
        output_dir.mkdir()
    nlp.to_disk(output_dir)
    print("Saved model to", output_dir)



In [ ]:
TEST_DATA=convert_dataturks_to_spacy("../input/testdata/output.json")

In [ ]:
# test the saved model
print("Loading from", output_dir)
nlp2 = spacy.load(output_dir)
for text, _ in TEST_DATA:
    doc = nlp2(text)
    print("Entities", [(ent.text, ent.label_) for ent in doc.ents])
#     print("Tokens", [(t.text, t.ent_type_, t.ent_iob) for t in doc])